# Déployez un modèle dans le cloud

### 1. Démarrage de la session Spark 

In [1]:
# L'exécution de cette cellule démarre l'application Spark 

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1692391169886_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Affichage des informations sur la session en cours et liens vers Spark UI 

In [2]:
%%info 

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1692391169886_0002,pyspark,idle,Link,Link,None,
3,application_1692391169886_0004,pyspark,idle,Link,Link,None,✔


### 2. Installation des packages

Les packages nécessaires ont été installé via l'étape de **bootstrap** à l'instanciation du serveur.

### 3. Import des librairies

In [3]:
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from pyspark.ml.feature import StandardScaler, PCA 
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4. Définition des PATH pour charger les images et enregistrer les résultats

Nous accédons directement à nos **données sur S3** comme si elles étaient **stockées localement**.

In [4]:
PATH = 's3://p8-data-bach-elena'
PATH_Data = PATH+'/Test'
PATH_Result = PATH+'/Results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://p8-data-bach-elena
PATH_Data:   s3://p8-data-bach-elena/Test
PATH_Result: s3://p8-data-bach-elena/Results

### 5. Traitement des données

Chargement des données

In [5]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://p8-data-bach...|2023-08-14 07:49:13|  7353|[FF D8 FF E0 00 1...|
|s3://p8-data-bach...|2023-08-14 07:49:13|  7350|[FF D8 FF E0 00 1...|
|s3://p8-data-bach...|2023-08-14 07:49:13|  7349|[FF D8 FF E0 00 1...|
|s3://p8-data-bach...|2023-08-14 07:49:13|  7348|[FF D8 FF E0 00 1...|
|s3://p8-data-bach...|2023-08-14 07:49:14|  7328|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

Je ne conserve que le **path** de l'image et j'ajoute une colonne contenant les **labels** de chaque image :

In [7]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+-----------------------------------------------------+----------+
|path                                                 |label     |
+-----------------------------------------------------+----------+
|s3://p8-data-bach-elena/Test/Watermelon/r_106_100.jpg|Watermelon|
|s3://p8-data-bach-elena/Test/Watermelon/r_109_100.jpg|Watermelon|
|s3://p8-data-bach-elena/Test/Watermelon/r_108_100.jpg|Watermelon|
|s3://p8-data-bach-elena/Test/Watermelon/r_107_100.jpg|Watermelon|
|s3://p8-data-bach-elena/Test/Watermelon/r_95_100.jpg |Watermelon|
+-----------------------------------------------------+----------+
only showing top 5 rows

None

Préparation du modèle

In [8]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [12]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Définition du processus de chargement des images et application de leur featurisation à travers l'utilisation de pandas UDF

In [13]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt1/yarn/usercache/livy/appcache/application_1692391169886_0004/container_1692391169886_0004_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py:392: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

Exécutions des actions d'extractions de features

In [14]:
features_df = images.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
print(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://p8-data-bach-elena/Results

In [16]:
features_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 6.Chargement des données enregistrées et validation du résultat

In [19]:
df = pd.read_parquet(PATH_Result, engine='pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                path  ...                                           features
0  s3://p8-data-bach-elena/Test/Watermelon/r_107_...  ...  [0.0033765617, 0.64489454, 0.014761818, 0.0, 1...
1  s3://p8-data-bach-elena/Test/Watermelon/r_70_1...  ...  [0.95478183, 0.05699287, 0.0, 0.0, 0.60643506,...
2  s3://p8-data-bach-elena/Test/Pineapple Mini/32...  ...  [0.0, 4.5893946, 0.0, 0.0, 0.0, 0.0, 0.0175827...
3  s3://p8-data-bach-elena/Test/Watermelon/284_10...  ...  [0.28515866, 0.2002849, 0.0, 0.027041124, 1.95...
4  s3://p8-data-bach-elena/Test/Pineapple Mini/32...  ...  [0.0, 4.5827346, 0.0, 0.0, 0.0, 0.0, 0.1010069...

[5 rows x 3 columns]

In [21]:
df.loc[0,'features'].shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(1280,)

In [22]:
df.shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(22688, 3)

### 7. PCA 

Je ne conserve que la colonne features

In [23]:
df_features = features_df.select(col("features"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Je transforme l'array en vecteur.

In [24]:
array_to_vector = udf(lambda l: Vectors.dense(l), VectorUDT())
df_features = df_features.withColumn("features_vectors", array_to_vector("features"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Je standardise mes nouvelles features.

In [25]:
scaler = StandardScaler(inputCol = "features_vectors", outputCol = "scaledFeatures", withMean = True, 
                        withStd = True).fit(df_features)
df_scaled = scaler.transform(df_features)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

J'applique mon ACP à 2 composantes.

In [26]:
n_components = 2
pca = PCA(k = n_components, inputCol = "scaledFeatures", outputCol = "pcaFeatures").fit(df_scaled)
df_pca = pca.transform(df_scaled)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Je définie un nouveau path pour stoké mes données.

In [27]:
PATH_pca = "s3://p8-data-bach-elena/PCA"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

J'enregistre mes données dans mon path au format parquet.

In [28]:
df_pca.write.mode("overwrite").parquet(PATH_pca)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Je charge mes données enregistrées dans le path.

In [29]:
df_pca = pd.read_parquet(PATH_pca, engine='pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

J'affiche le haut de mon dataframe.

In [31]:
df_pca.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                            features  ...                                        pcaFeatures
0  [0.65066034, 0.2302475, 0.0, 0.050948504, 0.87...  ...  {'type': 1, 'size': None, 'indices': None, 'va...
1  [0.036237378, 0.15497826, 0.0, 0.04870765, 0.8...  ...  {'type': 1, 'size': None, 'indices': None, 'va...
2  [0.015392984, 4.681176, 0.0, 0.0, 0.0, 0.0, 0....  ...  {'type': 1, 'size': None, 'indices': None, 'va...
3  [0.0, 4.519895, 0.0, 0.0, 0.0, 0.0, 0.327507, ...  ...  {'type': 1, 'size': None, 'indices': None, 'va...
4  [0.0, 4.8245773, 0.0, 0.0, 0.0, 0.0, 0.0933932...  ...  {'type': 1, 'size': None, 'indices': None, 'va...

[5 rows x 4 columns]